# Разрешение неоднозначности: WSD


In [22]:
import re
from collections import Counter
import os
from pprint import pprint
from pymystem3 import Mystem

In [23]:
m = Mystem()

2. Препроцессинг
Перед тем, как из множества контекстов получить набор признаков, нужно 
(1) убрать лишние символы
(2) убрать стоп-слова
(3) лемматизировать тексты

In [24]:
def load_stop_words(stop_filename):
	''' загрузить список стоп-слов из файла, одно слово на строке '''
	with open(stop_filename, encoding = 'utf-8') as f:
		stopwords = [w.strip() for w in f.readlines() if w.strip()]
	return set(stopwords)

stopw = "./stoplist_russian.txt"
print(load_stop_words(stopw))


{'ли', 'лишь', 'пока', 'свой', 'главный', 'над', 'примечательно', 'и', 'едва', 'со', 'каждый', 'я', 'еще', 'год', 'без', 'она', 'первый', 'с', 'по', 'по-прежнему', 'здесь', 'рамка', 'самый', 'восемь', 'уже', 'он', 'четыре', 'на', 'в', 'не', 'а', 'даже', 'этот', 'ещё', 'стать', 'во', 'что', 'после', 'два'}


In [25]:
def preprocessing(raw_text):
    clean_text = re.sub('\W+', ' ', raw_text) # \W = [^a-zA-Z0-9_]
    return clean_text

def lemmatize(input):
    return [lemma.strip() for lemma in m.lemmatize(preprocessing(input.lower())) if lemma.strip()]

text = "с виду воду, которая бьёт  рядом с французским городом Авен"
#m.lemmatize(text)

#r_text = "Он еле дождался, пока я доковыряюсь ключом в замке и, не снимая сапожек, прямиком пошёл в свою спальню."
#print(preprocessing(text))
print(lemmatize(text))

['с', 'вид', 'вода', 'который', 'бить', 'рядом', 'с', 'французский', 'город', 'авен']


In [26]:
def remove_stop_words(lemmas, stopwords):
    return ' '.join([word for word in lemmas if word not in stopwords])

#stop_words_filename = 'D:/GooDrive/0_2018_Edu_CL2/0_2018_Edu_HSE_Bak_CL2/WSD/WSD_Classifier/stoplist_russian.txt' 	# одно слово на строке
stop_words = load_stop_words(stopw)
raw_texts = ['Он', 'еле', 'дождался', 'пока', 'я', 'доковыряюсь', 'ключом', 'в', 'замке', 'и', 'не', 'снимая']
print(remove_stop_words(raw_texts, stop_words))

Он еле дождался доковыряюсь ключом замке снимая


# 1. Загрузка и адаптация корпуса
В качестве корпуса мы возьмем два файла, каждый из которых включает примерно 1000-1500 контекстов со словом "ключ", который доступен с сайта.  
Каждый контекст с новой строки.
Пусть примеры из файла spring.txt получают значение "0", а из файла key.txt значение 1

#положительная оценка
#отрицательная оценка
#текст твита
Напишем функцию, которая читает этот файл

In [27]:
# библиотека для поддержки различных кодировок при чтении/записи файлов
# вообще, все import лучше хранить в начале файла, но в IPython'е это не всегда удобно
import codecs

def load_files(filename):
    # список, где будут накапливаться результаты
    results = []
    
    # открываем файл как переменную inp_file.
    # Важно помнить, что в этой переменной хранится не сам текстовый файл, а своего рода указатель на него.
    # Файл еще предстоит прочитать.
    # Конструкция with следит за тем, чтобы переменная, которая после as, существовала только внутри блока.
    # Благодаря этому файл закроется сразу же после использования
    with(codecs.open(filename, encoding = 'utf-8')) as inp_file:
        # читаем первую строку "в никуда" (не сохраняем результат).
        # Первая строка у нас — это названия колонок, они нам не нужны
        inp_file.readline()
        # Для каждой строки в файле
        for line in inp_file:
            # откусываем концы строк
            line = line.strip('\r\n')
            # строка.count(подстрока) возвращает количество вхождений подстроки в строку
            if line.count('\t') != 1:
            # специальное ключевое слово для перехода к следующей итерации цикла
                continue
            # делим строку по табуляции и складываем в каждую из двух переменных очередное значение
            sence, context = line.split('\t')
            # добавляем кортеж (sence, context) в общий список результатов
            cx = lemmatize(context)
            cx = remove_stop_words(cx, stop_words)
            results.append((int(sence), cx))
    # возвращаем результаты в виде списка кортежей (sence, context)
    return results

In [28]:
# указываем абсолютный или относительный путь
data = load_files('./spring-key.txt')

In [29]:
# напечатаем первые 10 элементов нашего корпуса
for item in data[:10]:
    print (item[0], item[1])

0 столица то чтобы бить но стоять
0 вид вода который бить рядом французский город авен
0 у обледенелый пень бить незамерзающий
0 дно озеро бить тысяча хотя расшевеливать его поверхность
0 землянка навстречу ты бить грязевый
0 из глубокий расселина бить горячий окутывать туман край впадина
0 каравай стол вода голова плечо
0 место шлюз бить могучий
0 подниматься река то подземный переставать биться
0 бить день ночь белый попыхивать белый дымок


In [54]:
with codecs.open('./lemmatized_data.txt', mode='w', encoding='utf-8') as wr:
    wr.write('sense\tcontext(lemmatized)\r\n')
    for item in data:
        wr.write(str(item[0]) + '\t' + item[1] + "\r\n")

2. Перевод корпуса в векторы признаков, feature vectors
Наш корпус загружен, теперь можно подумать об обучении. Что требуется для классификации?

Каждый объект, который передается в классификатор, для обучения или нет, должен быть преобразован в набор признаков (вектор). Количество признаков — это размерность вектора, а заодно и размерность пространства признаков. Чем больше признаков, тем проще может быть отличить один объект от другого, но тем больше расходы на хранение и вычисление (представьте, что у вас 1 000 000 объектов (в обучающей выборке или уже на этапе предсказания), и мы прибавили всего один признак — тут же надо добавить 1 000 000 значений).

При классификации текста часто используется подход мешок слов (bag of words). Это значит, что каждый текст рассматривается как набор слов, которые встретились в этом тексте. С одной стороны, теряются связи между токенами, с другой — всё становится значительно проще.

Мы будем пользоваться именно этим подходом для нашей задачи.

In [31]:
# математика
import numpy

# разные классификаторы
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

# векторизация
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# деление корпуса train/test
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# Pipeline
from sklearn.pipeline import Pipeline

In [32]:
x_context = [item[1] for item in data]
y_labels = [item[0] for item in data]
print(x_context[:5])
print(y_labels[1])

['столица то чтобы бить но стоять', 'вид вода который бить рядом французский город авен', 'у обледенелый пень бить незамерзающий', 'дно озеро бить тысяча хотя расшевеливать его поверхность', 'землянка навстречу ты бить грязевый']
0


In [33]:
def tokenize(input):
    return input.split(' ')

In [34]:
classifiers = [('SVM', LinearSVC), ('Bayes', MultinomialNB)]
vectorizers = [('Counts', CountVectorizer), ('TfIdf', TfidfVectorizer)]

In [35]:
classifiers[0][1]

sklearn.svm.classes.LinearSVC

Хочется получить:

Оценку по всем сочетаниям классификатора и векторизатора
Препроцессинг
По результатам: построить график

In [67]:
def score_classifier(clf, metric='f1'):
    scores = cross_val_score(pipeline, numpy.asarray(x_context), numpy.asarray(y_labels), cv=5, scoring=metric)
    score = sum(scores) / len(scores)
    
    return score

In [92]:
clf = MultinomialNB()
pipeline = Pipeline(
[
  ('vectorizer',  TfidfVectorizer(tokenizer=tokenize)),
  ('classifier',  clf)
])

pipeline.fit(numpy.asarray(x_context), numpy.asarray(y_labels))

print([' - '.join([str(n), x_context[item]]) for item, n in enumerate(pipeline.predict(x_context))])

print(pipeline.predict([u'из ключа вода']))

print (score_classifier(pipeline))
print (score_classifier(pipeline, 'precision'))
print (score_classifier(pipeline, 'recall'))

['0 - столица то чтобы бить но стоять', '0 - вид вода который бить рядом французский город авен', '0 - у обледенелый пень бить незамерзающий', '0 - дно озеро бить тысяча хотя расшевеливать его поверхность', '0 - землянка навстречу ты бить грязевый', '0 - из глубокий расселина бить горячий окутывать туман край впадина', '0 - каравай стол вода голова плечо', '0 - место шлюз бить могучий', '0 - подниматься река то подземный переставать биться', '0 - бить день ночь белый попыхивать белый дымок', '0 - живой вода бить', '0 - запрокидываться бежать шуметь родниковый колода хрустальныим', '0 - фонтан вода громко зашуметь родниковый находить то', '0 - фонтан вода шуметь родниковый играть музыка', '0 - степной деревня мужик за неимение колодец пить какой то жидкий', '0 - тот сердце кой отворяться мертвый вода фонтан денно', '0 - чаша из который бить ртутный', '0 - изо весь истукан начинать бить который превосходить высота', '0 - источник извиваться стена', '0 - раздеваться тогда служитель отвора

0.872270756135
0.980701504918


In [38]:
for clf in classifiers:
    for vctr in vectorizers:
        pipeline = Pipeline([
    ('vectorizer', vctr[1]()),
    ('classifier', clf[1]()) ])
        print (clf[0], vctr[0], score_classifier(pipeline))

SVM Counts 0.927925550289
SVM TfIdf 0.934979642866
Bayes Counts 0.945543997664
Bayes TfIdf 0.925466632146


In [39]:
scores = cross_val_score(pipeline, numpy.asarray(x_context), numpy.asarray(y_labels), cv=5, scoring='f1')
score = sum(scores) / len(scores)

print (score)
print (scores)

0.925466632146
[ 0.91255962  0.92063492  0.92307692  0.93617021  0.93489149]


In [40]:
def score_classifier(clf, metric='f1'):
    scores = cross_val_score(pipeline, numpy.asarray(x_tweets), numpy.asarray(y_labels), cv=5, scoring=metric)
    score = sum(scores) / len(scores)
    
    return score

In [41]:
scores = cross_val_score(pipeline, numpy.asarray(x_context), numpy.asarray(y_labels), cv=5, scoring='f1')
score = sum(scores) / len(scores)

print (score)
print (scores)

0.925466632146
[ 0.91255962  0.92063492  0.92307692  0.93617021  0.93489149]


In [55]:
#import sklearn

from nltk.corpus import stopwords
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.model_selection import train_test_split

import numpy as np

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import os, sys, codecs

In [56]:
texts = [item[1] for item in data]
for i in range(10):
    print (texts[i])
    
len(texts)

print (3974 *16344 * 4)

столица то чтобы бить но стоять
вид вода который бить рядом французский город авен
у обледенелый пень бить незамерзающий
дно озеро бить тысяча хотя расшевеливать его поверхность
землянка навстречу ты бить грязевый
из глубокий расселина бить горячий окутывать туман край впадина
каравай стол вода голова плечо
место шлюз бить могучий
подниматься река то подземный переставать биться
бить день ночь белый попыхивать белый дымок
259804224


In [57]:
vectorizer = TfidfVectorizer()

texts = [item[1] for item in data]
vectorized = vectorizer.fit_transform(texts)
vectorized

print (vectorized[0], vectorized[0].shape)

  (0, 3897)	0.612887235558
  (0, 4053)	0.304502124418
  (0, 4449)	0.403195701632
  (0, 194)	0.226723384434
  (0, 2319)	0.324988460592
  (0, 3904)	0.460499648571 (1, 4582)


In [58]:
clf = RandomForestClassifier(n_estimators=250)
y_labels = [item[0] for item in data]
clf.fit(vectorized.toarray(), y_labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=250, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [59]:
#clf.predict(vectorizer.fit([u'все было ужасно']))
clf.predict(vectorized[0].toarray())
print (y_labels[0])

0


In [96]:
y_labels = [item[0] for item in data]
X_train, X_test, y_train, y_test = train_test_split(vectorized, y_labels, test_size=0.1, random_state=42)
len(y_labels)
#X_train.shape
#y_train.shape
X_test[0].getnnz()

8

In [61]:
names = vectorizer.get_feature_names()

  (0, 3023)	0.363052880909
  (0, 2348)	0.385357873902
  (0, 2274)	0.32806657826
  (0, 2728)	0.407662866895
  (0, 2391)	0.376877909129
  (0, 558)	0.407662866895
  (0, 487)	0.179094349693
  (0, 431)	0.324922243533
  (1, 4082)	0.640549897094
  (1, 1214)	0.215361492749
  (1, 4069)	0.576753331284
  (1, 4053)	0.458988944994
  (2, 3064)	0.614375913043
  (2, 4281)	0.614375913043
  (2, 3458)	0.439807872543
  (2, 194)	0.227274086061
  (3, 1799)	0.50286568655
  (3, 1609)	0.50286568655
  (3, 895)	0.50286568655
  (3, 1012)	0.416841152954
  (3, 1214)	0.117226980846
  (3, 1489)	0.232119779507
  (4, 4276)	0.436015925352
  (4, 1496)	0.355090050224
  (4, 2090)	0.398721606389
  :	:
  (239, 2887)	0.428529643998
  (239, 1214)	0.135746360625
  (239, 1524)	0.340344926072
  (239, 3568)	0.357001433414
  (240, 1191)	0.490509548821
  (240, 3274)	0.490509548821
  (240, 4109)	0.490509548821
  (240, 840)	0.379986197974
  (240, 2534)	0.234789821765
  (240, 1313)	0.28050913798
  (241, 480)	0.436157027902
  (241, 3054

In [62]:
items = clf.predict(X_test.toarray())
right = 0
for i in range(len(items)):
    if items[i] == y_test[i]:
        right += 1
        
print ('Total: %d\nRight: %d\nScore: %.3f' % (len(items), right, float(right) / len(items)))

words = {}
for i in range(len(clf.feature_importances_)):
    words[names[i]] = clf.feature_importances_[i]
    
for item in sorted(words, key = lambda word: words[word], reverse=True):
    print (item, words[item])

Total: 244
Right: 240
Score: 0.984
замок 0.182753439045
бить 0.0975511336164
вода 0.0678807578426
открывать 0.0344507709407
дверь 0.0231172754973
земля 0.0200687898101
из 0.0185540994625
чистый 0.0134211723159
звенеть 0.0128650513104
повертывать 0.01045551427
река 0.00935239330038
студеный 0.00859386158657
дно 0.00652658444657
повертываться 0.00488643164807
вставлять 0.00485397935612
горячий 0.00453714938957
холодный 0.00446662242086
жизнь 0.00408799948131
зажигание 0.00404359498311
где 0.0040264262788
запирать 0.00400125841246
карман 0.00398327542148
живой 0.00394815609889
отпирать 0.00392414502191
пить 0.003701409789
от 0.00364210921564
источник 0.00356851376342
теплый 0.00332125839827
щелкнуть 0.00330159425373
подходить 0.00322379302723
энергия 0.00306300400164
который 0.00304777291305
один 0.00299996854453
гора 0.0028907172811
родник 0.00287581107109
озеро 0.00286762279206
под 0.00276504276983
за 0.00262228934702
быть 0.00255009792429
связка 0.00254449888537
только 0.00248582327119

сад 7.89334973373e-05
басовый 7.88917813329e-05
ближневосточный 7.88121450469e-05
спасать 7.87955290148e-05
невидимый 7.87766490941e-05
рыться 7.86314774955e-05
хозяйский 7.86125982686e-05
нрав 7.85510688685e-05
воровать 7.84544019223e-05
покопошиться 7.83948826327e-05
случаться 7.82799179072e-05
заблокировать 7.82768409488e-05
разгадывание 7.81543424645e-05
зуммер 7.80779244941e-05
гаснуть 7.80333363949e-05
владеть 7.80316376593e-05
труба 7.80157760259e-05
honda 7.80091997967e-05
значит 7.78287395699e-05
возвышать 7.78011945282e-05
переставать 7.76855955421e-05
везде 7.76816022921e-05
прочее 7.76019421381e-05
прислушиваться 7.74971092178e-05
звонить 7.74710808338e-05
ружье 7.73921641875e-05
богатырь 7.73856547928e-05
посуда 7.73646928924e-05
подниматься 7.73393247199e-05
требоваться 7.72259286914e-05
громко 7.70589461162e-05
неневестный 7.70538484122e-05
могучий 7.69716880785e-05
низкий 7.66976297583e-05
часовой 7.65238535249e-05
пульсировать 7.64317618511e-05
обучать 7.6398283387e-05

покупка 7.09814568951e-06
дарование 7.06028323585e-06
благородный 7.03327590283e-06
условленный 7.02417703123e-06
призыв 7.02223969064e-06
примечать 7.0200713213e-06
искупаться 7.01409579583e-06
расщелина 7.00787566718e-06
адамыр 7.0026480833e-06
ssl 6.9935780774e-06
порадовать 6.96590341934e-06
наизнанку 6.96227566968e-06
чайка 6.93540400184e-06
молвить 6.93200723224e-06
заботиться 6.91748218025e-06
калиновый 6.91076914973e-06
племя 6.89846086104e-06
проедаться 6.89120768594e-06
достигаться 6.86761043381e-06
хлопать 6.82426647154e-06
козырь 6.823635283e-06
подарить 6.80769340335e-06
перо 6.79006743028e-06
стакан 6.7723918957e-06
успех 6.7460976399e-06
рубить 6.74261431481e-06
caguloe 6.73457138029e-06
восклицать 6.7214126728e-06
платонида 6.7197630195e-06
окрестный 6.65593692697e-06
алла 6.64493049733e-06
беспрестанный 6.58040403072e-06
обращать 6.57976256558e-06
снежный 6.57478906671e-06
политический 6.55714334693e-06
гроза 6.5406638825e-06
проливать 6.53444111615e-06
внешний 6.52332

In [66]:
#print data[10][0], data[10][1]
#items = clf.predict(vectorized[10])
#print items[0]
#print X_test
items = clf.predict(X_test.toarray())
right = 0
for i in range(len(items)):
    if items[i] == y_test[i]:
        right += 1
    #print items[i], y_test[i]
names = vectorizer.get_feature_names()
words = {}
print ('Total: %d\nRight: %d\nScore: %.3f' % (len(items), right, float(right) / len(items)))
for i in range(len(clf.feature_importances_)):
    words[names[i]] = clf.feature_importances_[i]
    
for item in sorted(words, key = lambda word: words[word], reverse=True):
    print (item, words[item])

Total: 244
Right: 240
Score: 0.984
замок 0.182753439045
бить 0.0975511336164
вода 0.0678807578426
открывать 0.0344507709407
дверь 0.0231172754973
земля 0.0200687898101
из 0.0185540994625
чистый 0.0134211723159
звенеть 0.0128650513104
повертывать 0.01045551427
река 0.00935239330038
студеный 0.00859386158657
дно 0.00652658444657
повертываться 0.00488643164807
вставлять 0.00485397935612
горячий 0.00453714938957
холодный 0.00446662242086
жизнь 0.00408799948131
зажигание 0.00404359498311
где 0.0040264262788
запирать 0.00400125841246
карман 0.00398327542148
живой 0.00394815609889
отпирать 0.00392414502191
пить 0.003701409789
от 0.00364210921564
источник 0.00356851376342
теплый 0.00332125839827
щелкнуть 0.00330159425373
подходить 0.00322379302723
энергия 0.00306300400164
который 0.00304777291305
один 0.00299996854453
гора 0.0028907172811
родник 0.00287581107109
озеро 0.00286762279206
под 0.00276504276983
за 0.00262228934702
быть 0.00255009792429
связка 0.00254449888537
только 0.00248582327119

прочее 7.76019421381e-05
прислушиваться 7.74971092178e-05
звонить 7.74710808338e-05
ружье 7.73921641875e-05
богатырь 7.73856547928e-05
посуда 7.73646928924e-05
подниматься 7.73393247199e-05
требоваться 7.72259286914e-05
громко 7.70589461162e-05
неневестный 7.70538484122e-05
могучий 7.69716880785e-05
низкий 7.66976297583e-05
часовой 7.65238535249e-05
пульсировать 7.64317618511e-05
обучать 7.6398283387e-05
трудоемкий 7.63642088311e-05
железистый 7.63162550704e-05
защищать 7.62723474605e-05
радостный 7.62108108686e-05
исследовать 7.61907182748e-05
никто 7.61713136184e-05
злоязычный 7.61310684052e-05
рахмановский 7.60571125161e-05
соседний 7.60431850558e-05
продавать 7.6038180761e-05
тыс 7.59921959237e-05
развязываться 7.59230603632e-05
безо 7.59196925628e-05
залив 7.58687522519e-05
гия 7.57938192161e-05
чайник 7.55902858166e-05
направление 7.52941061617e-05
получение 7.52563453223e-05
стрелка 7.52397361365e-05
фонетически 7.52127796535e-05
ступенька 7.5183243685e-05
нарзан 7.51676515902e-

иволга 1.09003161973e-05
конвоир 1.08925526052e-05
невеста 1.08755329642e-05
гимн 1.08732916044e-05
кефгир 1.08646437514e-05
возвышенность 1.0862433744e-05
история 1.08483840269e-05
титусвиль 1.0802526283e-05
любовный 1.07981821218e-05
борщовка 1.07872515624e-05
конфигурация 1.0780018192e-05
зарастать 1.0763181138e-05
полет 1.07491547139e-05
повторять 1.07466530962e-05
выкатиться 1.07191484243e-05
еда 1.07169704164e-05
поклоняться 1.07086061781e-05
омываться 1.06597243708e-05
атрибуция 1.06353872879e-05
скучный 1.06288041437e-05
пьетро 1.0618676106e-05
вскипать 1.06151724539e-05
книга 1.05861503741e-05
разгуливать 1.04420907681e-05
развеваться 1.04411966316e-05
павильон 1.04304199911e-05
восторг 1.04103661332e-05
умолкать 1.03898724852e-05
плоть 1.03672167583e-05
поверх 1.02941441571e-05
эпоха 1.02696383265e-05
кануть 1.02039434034e-05
батак 1.02027794939e-05
изшедый 1.01884473766e-05
глухой 1.01750124133e-05
ковырять 1.01359456213e-05
привлекательный 1.01272043765e-05
парк 1.012649861

пистолет 2.77631330998e-10
разглядывать 2.36361808844e-10
психологический 2.30562120049e-10
сие 2.26752792825e-10
насчет 2.15483420513e-10
формула 2.13128601962e-10
наковальня 2.12250218313e-10
деревян 2.05317850452e-10
дублироваться 2.03268877604e-10
скобка 1.75254640028e-10
динамик 1.73529342509e-10
внтуренний 1.67959502374e-10
евдоким 1.66010555641e-10
st 1.62653945785e-10
идентичность 1.50270132936e-10
детский 1.49879589931e-10
артель 1.45607953359e-10
взгляд 1.28309918476e-10
реально 1.22617831958e-10
вкладываться 1.19463698535e-10
дырочка 1.14508172835e-10
основа 1.14032055228e-10
азат 9.72103595275e-11
вопросительный 9.15988684998e-11
попивать 8.65179977746e-11
сид 8.62588857547e-11
дамский 8.43127170848e-11
александр 7.42159916285e-11
остальной 6.33461185495e-11
передыхать 5.36798232039e-11
делиться 0.0
встраиваться 0.0
постсинаптический 0.0
николаевна 0.0
анна 0.0
эди 0.0
серия 0.0
камышинский 0.0
борьба 0.0
intrepida 0.0
крип 0.0
кабаллистический 0.0
слесарь 0.0
подруга 0.0
к

In [63]:
names = vectorizer.get_feature_names()
len(names)

4582

In [65]:
print (len(data))
kf = KFold(3300, n_folds=10)
for train, test in kf:
    #print train, test
    X_train, X_test, y_train, y_test = vectorized[train], vectorized[test], y_labels[train], y_labels[test]

2432


IndexError: index (3299) out of range

In [80]:
clf = MultinomialNB()
pipeline = Pipeline(
[
  ('vectorizer',  TfidfVectorizer(tokenizer=tokenize)),
  ('classifier',  clf)
])

pipeline.fit(numpy.asarray(x_context), numpy.asarray(y_labels))
pipeline.predict(x_context)

pipeline.predict(test_set)

print (score_classifier(pipeline))
print (score_classifier(pipeline, 'precision'))
print (score_classifier(pipeline, 'recall'))

0.923030789281
0.872270756135
0.980701504918


In [ ]:
y_labels = [item[0] for item in data]
X_train, X_test, y_train, y_test = train_test_split(vectorized, y_labels, test_size=0.1, random_state=42)
len(y_labels)
#X_train.shape
#y_train.shape
X_test[0].getnnz()

In [79]:
test_set = ['открыть замок дверь', 'сильно сжать кулак врезаться кожа', 'пить чистый вода', 'лес весна холодный земля', 'набрать полный бутылка', 'положить карман', 'забыть дома не мочь зайти']



In [ ]:
items = clf.predict(X_test.toarray())
right = 0
for i in range(len(items)):
    if items[i] == y_test[i]:
        right += 1
    #print items[i], y_test[i]
names = vectorizer.get_feature_names()
words = {}
print ('Total: %d\nRight: %d\nScore: %.3f' % (len(items), right, float(right) / len(items)))
for i in range(len(clf.feature_importances_)):
    words[names[i]] = clf.feature_importances_[i]
    
for item in sorted(words, key = lambda word: words[word], reverse=True):
    print (item, words[item])